In [33]:
from __future__ import print_function

import requests

In [2]:
def do_request(service, a1=None, a2=None, a3=None, **kwargs):
    server = 'http://api.gbif.org/v1'
    params = ''
    for a in [a1, a2, a3]:
        if a is not None:
            params += '/' + a
    req = requests.get('%s/%s%s' % (server, service, params),
                       params=kwargs,
                       headers={'Content-Type': 'application/json'})
    if not req.ok:
        req.raise_for_status()
    return req.json()

In [24]:
req = do_request('species', 'search', q='bear')
print(req['count'])
req['results'][0]

19204


{u'authorship': u'',
 u'canonicalName': u'Ursus white bear',
 u'class': u'Mammalia',
 u'classKey': 106223020,
 u'datasetKey': u'fab88965-e69d-4491-a04d-e3198b626e52',
 u'descriptions': [],
 u'family': u'Ursidae',
 u'familyKey': 106657396,
 u'genus': u'Ursus',
 u'genusKey': 106658119,
 u'habitats': [],
 u'higherClassificationMap': {u'106148414': u'Metazoa',
  u'106151875': u'Carnivora',
  u'106223020': u'Mammalia',
  u'106522535': u'Chordata',
  u'106657396': u'Ursidae',
  u'106658119': u'Ursus'},
 u'key': 106189791,
 u'kingdom': u'Metazoa',
 u'kingdomKey': 106148414,
 u'nameType': u'SCINAME',
 u'nomenclaturalStatus': [],
 u'numDescendants': 0,
 u'numOccurrences': 0,
 u'order': u'Carnivora',
 u'orderKey': 106151875,
 u'parent': u'Ursus',
 u'parentKey': 106658119,
 u'phylum': u'Chordata',
 u'phylumKey': 106522535,
 u'rank': u'SPECIES',
 u'scientificName': u'Ursus sp. Shennongjia white bear',
 u'species': u'Ursus white bear',
 u'speciesKey': 106189791,
 u'synonym': False,
 u'taxonID': u'3

In [29]:
req = do_request('species', 'search', q='bear', rank='family')
print(req['count'])
bears = req['results'][0]
print(bears)

645
{u'nameType': u'WELLFORMED', u'family': u'Ursidae', u'nubKey': 9681, u'rank': u'FAMILY', u'phylum': u'Chordata', u'orderKey': 143045479, u'higherClassificationMap': {u'143045479': u'Carnivora', u'143035196': u'Chordata', u'116630539': u'Animalia', u'143045262': u'Mammalia'}, u'extinct': False, u'vernacularNames': [{u'language': u'eng', u'vernacularName': u'Bear'}], u'classKey': 143045262, u'habitats': [], u'familyKey': 110503853, u'kingdom': u'Animalia', u'parent': u'Carnivora', u'kingdomKey': 116630539, u'taxonID': u'1599257', u'canonicalName': u'Ursidae', u'key': 110503853, u'authorship': u'G. Fischer de Waldheim, 1817', u'nomenclaturalStatus': [], u'phylumKey': 143035196, u'class': u'Mammalia', u'numOccurrences': 0, u'synonym': False, u'datasetKey': u'cbb6498e-8927-405a-916b-576d00a6289b', u'threatStatuses': [], u'numDescendants': 35, u'scientificName': u'Ursidae G. Fischer de Waldheim, 1817', u'parentKey': 143045479, u'order': u'Carnivora', u'descriptions': [{u'description': u'

In [53]:
def get_all_records(rec_field, service, a1=None, a2=None, a3=None, **kwargs):
    records = []
    all_done = False
    offset = 0
    num_iter = 0
    while not all_done and num_iter < 10:
        req = do_request(service, a1=a1, a2=a2, a3=a3, offset=offset, **kwargs)
        all_done = req['endOfRecords']
        offset += req['limit']
        records.extend(req[rec_field])
        num_iter += 1
    return records

records = get_all_records('results', 'species', str(bears['nubKey']), 'children')

In [54]:
def get_leaves(nub):
    leaves = []
    recs = get_all_records('results', 'species', str(nub), 'children')
    if len(recs) == 0:
        return None
    for rec in recs:
        rec_leaves = get_leaves(rec['nubKey'])
        if rec_leaves is None:
            leaves.append(rec)
        else:
            print(len(rec_leaves))
            leaves.extend(rec_leaves)
    return leaves

In [55]:
leaves = get_leaves(bears['nubKey'])

5
1
2
2
2
8
2
2
3
2
2
4
16
16
7
44


In [80]:
print(leaves[0])
#[leaf['nameType'] for leaf in leaves]
for rec in leaves:
    print(rec['scientificName'], rec['rank'])
    vernaculars = do_request('species', str(rec['nubKey']), 'vernacularNames', language='en')['results']
    for vernacular in vernaculars:
        if vernacular['language'] == 'eng':
            print('   ' + vernacular['vernacularName'])
            break
    profile = do_request('species', str(rec['nubKey']), 'speciesProfiles')['results']
    if len(profile) == 0:
        continue
    my_profile = profile[0]  # we take the first
    print(my_profile)

{u'origin': u'SOURCE', u'nameType': u'WELLFORMED', u'family': u'Ursidae', u'nubKey': 4833637, u'lastInterpreted': u'2014-11-03T15:22:26.302+0000', u'accordingTo': u'Interim Register of Marine and Nonmarine Genera', u'rank': u'GENUS', u'phylum': u'Chordata', u'orderKey': 732, u'taxonomicStatus': u'ACCEPTED', u'issues': [], u'classKey': 359, u'familyKey': 9681, u'kingdom': u'Animalia', u'parent': u'Ursidae', u'kingdomKey': 1, u'taxonID': u'101710437', u'genusKey': 4833637, u'canonicalName': u'Agriarctos', u'key': 4833637, u'authorship': u'Kretzoi, 1942', u'nomenclaturalStatus': [], u'phylumKey': 44, u'class': u'Mammalia', u'synonym': False, u'datasetKey': u'd7dddbf4-2cf0-4f39-9b2a-bb099caae36c', u'publishedIn': u'F\xf6ldt. K\xf6zl. , Budapest, 72, 257, 350.', u'modified': u'2013-02-08T03:15:41.847+0000', u'numDescendants': 0, u'scientificName': u'Agriarctos Kretzoi, 1942', u'genus': u'Agriarctos', u'parentKey': 9681, u'order': u'Carnivora'}
Agriarctos Kretzoi, 1942 GENUS
{u'sourceTaxonKe